# 2. Feature Engineering


## Customer Demographics Features

1. `Age`
**Definition:**  
`Age = Current_Year - Year_Birth`  
**Example:**  
2024 - 1970 = **54**

2. `Is_Parent`
**Definition:**  
`Is_Parent = 1 if (Kidhome + Teenhome) > 0 else 0`

3. `Total_Kids`
**Definition:**  
`Total_Kids = Kidhome + Teenhome`

4. `Marital_Status_Simplified`
**Definition:**  
Group marital status into broader categories:
- `"Married"` or `"Together"` → **Partnered**  
- All others (e.g., `"Single"`, `"Divorced"`, `"Widow"`, etc.) → **Single**


## Purchase Behavior Features

1. `Total_Spent`
**Definition:**  
`Total_Spent = MntWines + MntFruits + MntMeatProducts + MntFishProducts + MntSweetProducts + MntGoldProds`

2. `Spending_per_Category`
**Definition:**  
Share of spending per category (example for Wine and Meat):  
- `Wine_Share = MntWines / Total_Spent`  
- `Meat_Share = MntMeatProducts / Total_Spent`  
*(Apply similarly for other product categories)*

3. `Average_Spend_per_Purchase`
**Definition:**  
`Avg_Spend_per_Purchase = Total_Spent / (NumDealsPurchases + NumWebPurchases + NumCatalogPurchases + NumStorePurchases)`

4. `Deal_Dependency`
**Definition:**  
`Deal_Rate = NumDealsPurchases / Total_Purchases`

## Channel Engagement Features

1. `Total_Purchases`
**Definition:**  
`Total_Purchases = NumWebPurchases + NumCatalogPurchases + NumStorePurchases`

2. `Web_Engagement`
**Definitions:**  
- `Web_Purchase_Share = NumWebPurchases / Total_Purchases`  
- `Web_Visit_to_Purchase_Ratio = NumWebVisitsMonth / (NumWebPurchases + 1)`

3. `Is_Online_Buyer`
**Definition:**  
`Is_Online_Buyer = 1 if NumWebPurchases > NumStorePurchases else 0`

## Temporal Features

1. `Customer_Since_Days`
**Definition:**  
`Customer_Since_Days = Today - Dt_Customer`  
*(Convert `Dt_Customer` to datetime and subtract from current date)*

2. `Recent_Activity`
**Definition:**  
`Is_Active = 1 if Recency < 30 else 0`

3. `Customer_Lifetime_Spent_per_Day`
**Definition:**  
`Lifetime_Spend_per_Day = Total_Spent / Customer_Since_Days`


## Campaign Response Features

1. `Total_Accepted_Campaigns`
**Definition:**  
Sum of all campaign columns:  
`Total_Accepted_Campaigns = AcceptedCmp1 + AcceptedCmp2 + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5`

2. `Campaign_Response_Rate`
**Definition:**  
`Campaign_Response_Rate = Total_Accepted_Campaigns / 5`

3. `Is_Responder`
**Definition:**  
`Is_Responder = 1 if Total_Accepted_Campaigns > 0 or Response == 1 else 0`

In [62]:
import pandas as pd
from datetime import datetime
from datetime import date
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

# set up to view all the info of the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [30]:
#load data
#df = pd.read_csv('/Users/adityaagarwal/Library/CloudStorage/OneDrive-NortheasternUniversity/Jupyter Notebook/Projects/CRM-Analysis-for-Marketing-data/marketing data/marketing_data.csv')
df = pd.read_csv('D:/OneDrive - Northeastern University/Jupyter Notebook/Data Science Projects/CRM-Analysis-for-Marketing-data/marketing data/clean_marketing_data.csv')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,8996,1957,PhD,Married,51381,2,1,11/19/12,4,230,42,192,49,37,53,12,7,2,8,9,0,0,0,0,0,0,0,GER
1,1994,1983,Graduation,Married,51381,1,0,11/15/13,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,0,US
2,3769,1972,PhD,Together,51381,1,0,3/2/14,17,25,1,13,0,0,3,1,1,0,3,7,0,0,0,0,0,0,0,AUS
3,5255,1986,Graduation,Single,51381,1,0,2/20/13,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,0,AUS
4,8268,1961,PhD,Married,51381,0,1,7/11/13,23,352,0,27,10,0,15,3,6,1,7,6,0,0,0,0,0,0,0,CA


In [31]:
df.dtypes

ID                      int64
Year_Birth              int64
Education              object
Marital_Status         object
Income                  int64
Kidhome                 int64
Teenhome                int64
Dt_Customer            object
Recency                 int64
MntWines                int64
MntFruits               int64
MntMeatProducts         int64
MntFishProducts         int64
MntSweetProducts        int64
MntGoldProds            int64
NumDealsPurchases       int64
NumWebPurchases         int64
NumCatalogPurchases     int64
NumStorePurchases       int64
NumWebVisitsMonth       int64
AcceptedCmp3            int64
AcceptedCmp4            int64
AcceptedCmp5            int64
AcceptedCmp1            int64
AcceptedCmp2            int64
Response                int64
Complain                int64
Country                object
dtype: object

In [32]:
# Convert Dt_Customer to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')

## --- Feature Engineering ---

#### 1. Age

In [33]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,8996,1957,PhD,Married,51381,2,1,2012-11-19,4,230,42,192,49,37,53,12,7,2,8,9,0,0,0,0,0,0,0,GER
1,1994,1983,Graduation,Married,51381,1,0,2013-11-15,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,0,US
2,3769,1972,PhD,Together,51381,1,0,2014-03-02,17,25,1,13,0,0,3,1,1,0,3,7,0,0,0,0,0,0,0,AUS
3,5255,1986,Graduation,Single,51381,1,0,2013-02-20,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,0,AUS
4,8268,1961,PhD,Married,51381,0,1,2013-07-11,23,352,0,27,10,0,15,3,6,1,7,6,0,0,0,0,0,0,0,CA


In [34]:
date.today()

datetime.date(2025, 5, 8)

In [36]:
df['Age'] = datetime.now().year - df['Year_Birth']

#### 2. is_parent

In [39]:
df['Is_Parent'] = ((df['Kidhome'] + df['Teenhome']) > 0).astype(int)

df['Is_Parent'].value_counts().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of Is_Parent
1    1602
0     638
Name: count, dtype: int64>

#### 3. Total_Kids

In [40]:
df['Total_Kids'] = df['Kidhome'] + df['Teenhome']

#### 4. Marital Status Simplified

In [43]:
df['Marital_Status_Simplified'] = df['Marital_Status'].replace({
    'Married': 'Partnered',
    'Together': 'Partnered',
    'Single': 'Single',
    'Divorced': 'Single',
    'Widow': 'Single',
    'Alone': 'Single',
    'Absurd': 'Single',
    'YOLO': 'Single'
})

df['Marital_Status_Simplified'].value_counts().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of Marital_Status_Simplified
Partnered    1444
Single        796
Name: count, dtype: int64>

#### 1. Total_Spent

In [44]:
spend_cols = ['MntWines', 'MntFruits', 'MntMeatProducts',
              'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_Spent'] = df[spend_cols].sum(axis=1)

#### 2. Spending per Category Share

In [47]:
for col in spend_cols:
    share_col = col + '_Share'
    df[share_col] = df[col] / df['Total_Spent'].replace(0, 1)

df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Age,Is_Parent,Total_Kids,Marital_Status_Simplified,Total_Spent,MntWines_Share,MntFruits_Share,MntMeatProducts_Share,MntFishProducts_Share,MntSweetProducts_Share,MntGoldProds_Share
0,8996,1957,PhD,Married,51381,2,1,2012-11-19,4,230,42,192,49,37,53,12,7,2,8,9,0,0,0,0,0,0,0,GER,68,1,3,Partnered,603,0.381426,0.069652,0.318408,0.081260,0.061360,0.087894
1,1994,1983,Graduation,Married,51381,1,0,2013-11-15,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,0,US,42,1,1,Partnered,19,0.263158,0.263158,0.315789,0.000000,0.105263,0.052632
2,3769,1972,PhD,Together,51381,1,0,2014-03-02,17,25,1,13,0,0,3,1,1,0,3,7,0,0,0,0,0,0,0,AUS,53,1,1,Partnered,42,0.595238,0.023810,0.309524,0.000000,0.000000,0.071429
3,5255,1986,Graduation,Single,51381,1,0,2013-02-20,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,0,AUS,39,1,1,Single,637,0.007849,0.001570,0.004710,0.004710,0.412873,0.568289
4,8268,1961,PhD,Married,51381,0,1,2013-07-11,23,352,0,27,10,0,15,3,6,1,7,6,0,0,0,0,0,0,0,CA,64,1,1,Partnered,404,0.871287,0.000000,0.066832,0.024752,0.000000,0.037129


#### 3. Average Spend per Purchase

In [48]:
df['Total_Purchases'] = df[['NumDealsPurchases', 'NumWebPurchases',
                            'NumCatalogPurchases', 'NumStorePurchases']].sum(axis=1)
df['Avg_Spend_per_Purchase'] = df['Total_Spent'] / df['Total_Purchases'].replace(0, 1)

#### 4. Deal Dependency

In [49]:
df['Deal_Rate'] = df['NumDealsPurchases'] / df['Total_Purchases'].replace(0, 1)

#### 1. Web Engagement

In [50]:
df['Web_Purchase_Share'] = df['NumWebPurchases'] / df['Total_Purchases'].replace(0, 1)
df['Web_Visit_to_Purchase_Ratio'] = df['NumWebVisitsMonth'] / (df['NumWebPurchases'] + 1)

#### 2. Is Online Buyer

In [51]:
df['Is_Online_Buyer'] = (df['NumWebPurchases'] > df['NumStorePurchases']).astype(int)

#### 1. Customer Since Days

In [52]:
df['Customer_Since_Days'] = (pd.to_datetime('2024-01-01') - df['Dt_Customer']).dt.days

#### 2. Is Active (based on Recency)

In [53]:
df['Is_Active'] = (df['Recency'] < 30).astype(int)

#### 3. Lifetime Spend per Day

In [54]:
df['Lifetime_Spend_per_Day'] = df['Total_Spent'] / df['Customer_Since_Days'].replace(0, 1)

#### 1. Total Accepted Campaigns

In [55]:
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df['Total_Accepted_Campaigns'] = df[campaign_cols].sum(axis=1)

#### 2. Campaign Response Rate

In [57]:
df['Campaign_Response_Rate'] = df['Total_Accepted_Campaigns'] / 5

#### 3. Is Responder

In [ ]:
df['Is_Responder'] = ((df['Total_Accepted_Campaigns'] > 0) | (df['Response'] == 1)).astype(int)

In [59]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Age,Is_Parent,Total_Kids,Marital_Status_Simplified,Total_Spent,MntWines_Share,MntFruits_Share,MntMeatProducts_Share,MntFishProducts_Share,MntSweetProducts_Share,MntGoldProds_Share,Total_Purchases,Avg_Spend_per_Purchase,Deal_Rate,Web_Purchase_Share,Web_Visit_to_Purchase_Ratio,Is_Online_Buyer,Customer_Since_Days,Is_Active,Lifetime_Spend_per_Day,Total_Accepted_Campaigns,Campaign_Response_Rate
0,8996,1957,PhD,Married,51381,2,1,2012-11-19,4,230,42,192,49,37,53,12,7,2,8,9,0,0,0,0,0,0,0,GER,68,1,3,Partnered,603,0.381426,0.069652,0.318408,0.081260,0.061360,0.087894,29,20.793103,0.413793,0.241379,1.125000,0,4060,1,0.148522,0,0.0
1,1994,1983,Graduation,Married,51381,1,0,2013-11-15,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,0,US,42,1,1,Partnered,19,0.263158,0.263158,0.315789,0.000000,0.105263,0.052632,4,4.750000,0.250000,0.250000,3.500000,0,3699,1,0.005137,0,0.0
2,3769,1972,PhD,Together,51381,1,0,2014-03-02,17,25,1,13,0,0,3,1,1,0,3,7,0,0,0,0,0,0,0,AUS,53,1,1,Partnered,42,0.595238,0.023810,0.309524,0.000000,0.000000,0.071429,5,8.400000,0.200000,0.200000,3.500000,0,3592,1,0.011693,0,0.0
3,5255,1986,Graduation,Single,51381,1,0,2013-02-20,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,0,AUS,39,1,1,Single,637,0.007849,0.001570,0.004710,0.004710,0.412873,0.568289,27,23.592593,0.000000,1.000000,0.035714,1,3967,1,0.160575,0,0.0
4,8268,1961,PhD,Married,51381,0,1,2013-07-11,23,352,0,27,10,0,15,3,6,1,7,6,0,0,0,0,0,0,0,CA,64,1,1,Partnered,404,0.871287,0.000000,0.066832,0.024752,0.000000,0.037129,17,23.764706,0.176471,0.352941,0.857143,0,3826,1,0.105593,0,0.0


In [63]:
os.chdir(r'D:\OneDrive - Northeastern University\Jupyter Notebook\Data Science Projects\CRM-Analysis-for-Marketing-data\marketing data')
df.to_csv("marketing_data_with_features.csv", index = False)